<a href="https://colab.research.google.com/github/escape13/deepfried/blob/master/IntroTorch/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch import optim
import matplotlib.pyplot as plt

In [2]:
from google.colab import files
print("Please pload your kaggle key:")
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/kaggle.json
! kaggle datasets download -d chetankv/dogs-cats-images
! unzip -q dogs-cats-images.zip
! rm -R 'dog vs cat'
! mv dataset/training_set dataset/train 
! mv dataset/test_set dataset/test

Please pload your kaggle key:


Saving kaggle.json to kaggle.json
 97% 422M/435M [00:12<00:00, 90.4MB/s]
100% 435M/435M [00:12<00:00, 36.1MB/s]


In [3]:
data_dir = 'dataset'

train_transforms = transforms.Compose([transforms.ToTensor(), transforms.Resize(250), transforms.RandomResizedCrop(224), transforms.RandomVerticalFlip(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.ToTensor(), transforms.Resize(225), transforms.CenterCrop(224), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [4]:
model = models.densenet121(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [5]:
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                                        ('fc1', nn.Linear(1024, 500)),
                                        ('relu', nn.ReLU()),
                                        ('fc2', nn.Linear(500, 2)),
                                        ('output', nn.LogSoftmax(dim=1))
                                        ]))
model.classifier = classifier

# Recognizing dogs and cats

In [6]:
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.required_grad = False

classifier = nn.Sequential(nn.Linear(1024, 256),
                           nn.ReLU(),
                           nn.Dropout(p=0.2),
                           nn.Linear(256, 2),
                           nn.LogSoftmax(dim=1))

model.classifier = classifier

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

In [9]:
epochs = 1
steps = 0
running_loss = 0
every = 5

for e in range(epochs):
    for inputs, labels in trainloader:
        optimizer.zero_grad()
        steps += 1
        output = model.forward(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if steps % every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    output = model.forward(inputs)
                    loss = criterion(output, labels)
                    test_loss += loss.item()

                    ps = torch.exp(output)
                    top_ps, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

            print(f"Epoch {e+1}/{epochs}.."
                f"Train loss: {running_loss/every:.3f}.."
                f"Test loss: {test_loss/len(testloader):.3f}.."
                f"Accuracy: {accuracy/len(testloader):.3f}")
            running_loss=0
            model.train()

Epoch 1/1..Train loss: 0.340..Test loss: 0.152..Accuracy: 0.974
Epoch 1/1..Train loss: 0.306..Test loss: 0.102..Accuracy: 0.977
Epoch 1/1..Train loss: 0.250..Test loss: 0.106..Accuracy: 0.962
Epoch 1/1..Train loss: 0.234..Test loss: 0.057..Accuracy: 0.984
Epoch 1/1..Train loss: 0.269..Test loss: 0.069..Accuracy: 0.976
Epoch 1/1..Train loss: 0.234..Test loss: 0.052..Accuracy: 0.986


KeyboardInterrupt: ignored